# Permit

Permit 是一个访问控制平台，使用 RBAC、ABAC 和 ReBAC 等各种模型提供细粒度的实时权限管理。它使组织能够在应用程序中强制执行动态策略，确保只有授权用户才能访问特定资源。

## 概述

此包提供了两个用于使用 Permit 进行 JWT 验证和权限检查的 Langchain 工具：

* LangchainJWTValidationTool：根据 JWKS 端点验证 JWT 令牌

* LangchainPermissionsCheckTool：使用 Permit 检查用户权限


## 设置

设置以下环境变量：

```bash
PERMIT_API_KEY=your_permit_api_key
JWKS_URL=your_jwks_endpoint_url
PERMIT_PDP_URL=your_permit_pdp_url  # 本地开发通常是 http://localhost:7766 或您的实际部署地址
```

确保您的 PDP（策略决策点）正在 PERMIT_PDP_URL 上运行。
有关策略设置以及如何启动 PDP 容器的详细信息，请参阅 [Permit 文档](https://docs.permit.io/concepts/pdp/overview/)。

### 凭证

```bash
PERMIT_API_KEY=
JWKS_URL=your_jwks_endpoint_url # 或你的部署 URL
PERMIT_PDP_URL=your_pdp_url # 或你的部署 URL
TEST_JWT_TOKEN= # 用于快速测试
```

在最先进的观察能力方面，设置 [LangSmith](https://smith.langchain.com/) 也是有帮助的（但非必需的）：

## 实例化

### JWT 验证工具
JWT 验证工具使用 JWKS (JSON Web Key Set) 端点来验证 JWT 令牌。

```python
from langchain_permit.tools import LangchainJWTValidationTool

# 初始化工具
jwt_validator = LangchainJWTValidationTool(
    jwks_url=#your url endpoint
)
```

### 配置选项
您可以使用以下任一方式初始化工具：

* JWKS URL
* 直接 JWKS JSON 数据
* 环境变量 (JWKS_URL)

```python
# 使用直接 JWKS JSON
jwt_validator = LangchainJWTValidationTool(
    jwks_json={
        "keys": [
            {
                "kid": "key-id",
                "kty": "RSA",
                ...
            }
        ]
    }
)
```

### 权限检查工具
权限检查工具集成了 Permit.io，用于验证用户对资源的权限。

```python
from permit import Permit
from langchain_permit.tools import LangchainPermissionsCheckTool

# 初始化 Permit 客户端
permit_client = Permit(
    token="your_permit_api_key",
    pdp=# Your PDP URL
)

# 初始化工具
permissions_checker = LangchainPermissionsCheckTool(
    permit=permit_client
)
```

本文档展示了这两个工具的关键功能和使用模式。

## 调用

### [直接使用参数调用](https://docs.permit.io/)

### JWT 验证工具

```python
# 验证一个 token
async def validate_token():
    claims = await jwt_validator._arun(
        "..."  # 你的 JWT token
    )
    print("已验证 Claims:", claims)
```

### 权限检查工具

```python
# 检查权限
async def check_user_permission():
    result = await permissions_checker._arun(
        user={
            "key": "user-123",
            "firstName": "John"
        },
        action="read",
        resource={
            "type": "Document",
            "tenant": "default"
        }
    )
    print("权限已授予:", result)
```

#### 输入格式
权限检查器接受不同的输入格式：

1. 用户简单字符串（将转换为用户 key）：

```python
result = await permissions_checker._arun(
    user="user-123",
    action="read",
    resource="Document"
)
```

2. 用户完整对象：

```python
result = await permissions_checker._arun(
    user={
        "key": "user-123",
        "firstName": "John",
        "lastName": "Doe",
        "email": "john@example.com",
        "attributes": {"department": "IT"}
    },
    action="read",
    resource={
        "type": "Document",
        "key": "doc-123",
        "tenant": "techcorp",
        "attributes": {"confidentiality": "high"}
    }
)
```

### [Invoke with ToolCall](https://docs.permit.io/)

(待办)

## 链式调用

- TODO: 添加用户问题和运行单元格

我们可以通过先将工具绑定到[工具调用模型](https://docs.permit.io/)，然后调用它来使用链式调用中的工具：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

### 附加演示脚本

要获取完整的可运行演示，请查看此 [仓库](https://github.com/permitio/langchain-permit) 中的 `/langchain_permit/examples/demo_scripts` 文件夹。您会找到：

* demo\_jwt\_validation.py – 一个快速脚本，展示如何使用 LangchainJWTValidationTool 验证 JWT。

* demo\_permissions\_check.py – 一个脚本，使用 LangchainPermissionsCheckTool 执行 Permit.io 权限检查。

只需运行 `python demo_jwt_validation.py` 或 `python demo_permissions_check.py`（在设置好环境变量后），即可看到这些工具的实际应用。

## API 参考

如需了解 Permit 所有功能和配置的详细文档，请前往 API 参考：https://docs.permit.io/